## Running and Processing a SWMMIN Simulation

In this notebook, we go over an example of running and processing a SWMMIN simulation created by either the ```Conversion_to_SWMMIN.ipynb``` notebook or the ```Convert_to_SWMMIN()``` method of the ```SWMMIN_sim()``` class.  

The execution of the SWMM simulation is done using PySWMM [[1]](#references).  

After execution, the output is processed and a few visualization and post-processing tools are showcased.

### Importing Required Packages

In [1]:
import pyswmm
import re
import pandas as pd
import numpy as np
import datetime
import matplotlib
import matplotlib.pyplot as plt
import pathlib

### Specifying SWMMIN input file
In this notebook the SWMMIN input file must be input directly.  
In the GOSWMMIN package, the SWMMIN_sim() class contains the input file path and the file can be executed directly using ```SWMMIN_sim().Run_SWMMIN()```

In [2]:
 # Replace with appropriate path and filename
directory=pathlib.Path('../Networks/Linear Network/')
filename=pathlib.Path('Linear_Network_SWMMIN_100m.inp')
name_only='_'.join(str(filename).split('_')[0:2])
print("Selected File: ",name_only)
path=directory/filename

Selected File:  Linear_Network


##### Open the SWMM simulation using PySWMM
Specifying the path for the input file and the future/current output file (.out) where the results are stored

In [3]:
sim=pyswmm.Simulation (str(path), outputfile=str(path.with_suffix(".out")))

##### Read the IDs of demand nodes (Notebook only)
Find the list of ids for demand nodes by identifying user storage nodes. In the GOSWMMIN package, this information is stored in the object

In [4]:
# Load the nodes object from the simulation
nodes=pyswmm.nodes.Nodes(sim)
# Initialize list of tank IDs
tankids=[]
# Loop through all nodes and find the tank IDs that start with StorageforNode
for node in nodes:
    if re.search('StorageforNode',node.nodeid):
        tankids.append(node.nodeid)

# Extract the demand node IDs from the tank IDs
demand_node_ids=[x[14:] for x in tankids]
# Create the withdrawal and consumption outlet IDs
Withdraw_ids = ["Outlet" + x for x in demand_node_ids]
Consume_ids = ["DemandOutlet" + x for x in demand_node_ids]

##### Run the SWMMIN simulation using PySWMM 

Using PySWMM, execute the input file and display progress updates and a final value for the continuity error.  
Tis cell can be skipped if the input file has already been run before and a .out file already exists with the intended simulation's results

In [5]:
stp=0       #steps counter
every=1000  #Interval of printing current time

# runs the simulation step by step
with sim as sim:
    system_routing = pyswmm.SystemStats(sim)
    for step in sim:
        if stp%every==0:
            print('Current Simulation Time is >> ',sim.current_time,", ",round(sim.percent_complete*100,1),"% Complete")

        stp+=1
        pass
    sim._model.swmm_end()
    print("Continuity Error: ",sim.flow_routing_error,"%\n")

Current Simulation Time is >>  2022-05-18 00:00:00 ,  0.0 % Complete
Current Simulation Time is >>  2022-05-18 00:16:40 ,  1.2 % Complete
Current Simulation Time is >>  2022-05-18 00:33:20 ,  2.3 % Complete
Current Simulation Time is >>  2022-05-18 00:50:00 ,  3.5 % Complete
Current Simulation Time is >>  2022-05-18 01:06:40 ,  4.6 % Complete
Current Simulation Time is >>  2022-05-18 01:23:20 ,  5.8 % Complete
Current Simulation Time is >>  2022-05-18 01:40:00 ,  6.9 % Complete
Current Simulation Time is >>  2022-05-18 01:56:40 ,  8.1 % Complete
Current Simulation Time is >>  2022-05-18 02:13:20 ,  9.3 % Complete
Current Simulation Time is >>  2022-05-18 02:30:00 ,  10.4 % Complete
Current Simulation Time is >>  2022-05-18 02:46:40 ,  11.6 % Complete
Current Simulation Time is >>  2022-05-18 03:03:20 ,  12.7 % Complete
Current Simulation Time is >>  2022-05-18 03:20:00 ,  13.9 % Complete
Current Simulation Time is >>  2022-05-18 03:36:40 ,  15.1 % Complete
Current Simulation Time is >>

In [6]:
WithdrawRates=pd.DataFrame()   #Empty Dataframe to store water depth in tanks
ConsumeRates=pd.DataFrame()   #Empty Dataframe to store water depth in nodes
swtch=True                   # switch variable for upcoming condition

# Reads the output file created above
with pyswmm.Output(str(path.with_suffix(".out"))) as out:
    # loops through each node in output file
    for link in out.links:

        # One time only. Gets the timesteps (the keys in the output series dictionary) and stores them to be used as index
        if swtch:
        # node_series produces a dictionary with the keys corresponding to timestamps and values contain the value of the selected variable (FLOW_RATE) at each timestamp
            index = pd.Series(out.link_series(link, 'Flow_rate').keys())
            WithdrawRates.loc[:,"time"]=index
            ConsumeRates.loc[:,"time"]=index
            swtch=False
        
        # If node id is in the prepared list of demand nodes (tanks)
        if link in Withdraw_ids:
            WithdrawRates.loc[:,link] = out.link_series(link, 'Flow_rate').values()  
        elif link in Consume_ids:
            ConsumeRates.loc[:,link]= out.link_series(link, 'Flow_rate').values()

AttributeError: module 'pyswmm' has no attribute 'Output'

In [37]:
Withdraw_by_Day = WithdrawRates.groupby(pd.Grouper(key = 'time', freq = 'D'))
last_day_W = Withdraw_by_Day.get_group(WithdrawRates['time'].max().floor('D'))

Consumer_by_Day = ConsumeRates.groupby(pd.Grouper(key = 'time', freq = 'D'))
last_day_C = Consumer_by_Day.get_group(ConsumeRates['time'].max().floor('D'))

Daily_Totals_W = WithdrawRates.groupby(pd.Grouper(key = 'time', freq = 'D')).sum().T*10/1000
Daily_Totals_C = ConsumeRates.groupby(pd.Grouper(key = 'time', freq = 'D')).sum().T.set_index(Daily_Totals_W.index)*10/1000

In [ ]:
Daily_Satis_W = pd.DataFrame(columns=Daily_Totals_W.columns, index = Daily_Totals_W.index)
Daily_Satis_C = pd.DataFrame(columns = Daily_Totals_C.columns, index = Daily_Totals_W.index)
for column in range(len(Daily_Totals_W.columns)):
        Daily_Satis_W.iloc[:,column] = Daily_Totals_W.iloc[:,column] / demand_volumes['Volume']*100
        Daily_Satis_C.iloc[:,column] = Daily_Totals_C.iloc[:,column] / demand_volumes['Volume']*100

# WATER BALANCE TOOL
# Effective Supply Duration

## References